# Structured Outputs with OpenAI APIs

Reference blog: https://openai.com/index/introducing-structured-outputs-in-the-api/

Reference documentation: https://platform.openai.com/docs/guides/structured-outputs/introduction

In [ ]:
%%capture
# update or install the necessary libraries
!pip install --upgrade openai
!pip install --upgrade python-dotenv

In [ ]:
from pydantic import BaseModel
from openai import OpenAI
from google.colab import userdata

# client session
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

## Simple Example of Structured Outputs

In [ ]:
# Pydantic object
class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

# call the model
completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."},
    ],
    response_format=CalendarEvent,
)

event = completion.choices[0].message.parsed

In [ ]:
event

CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob'])

## Structuring Chain-of-Thought Responses

In [ ]:
class Step(BaseModel):
    explanation: str
    output: str

class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
        {"role": "user", "content": "how can I solve 8x + 7 = -23"}
    ],
    response_format=MathReasoning,
)

math_reasoning = completion.choices[0].message.parsed

In [ ]:
math_reasoning

MathReasoning(steps=[Step(explanation='Begin by isolating the term with the variable. To do this, subtract 7 from both sides of the equation.', output='8x + 7 - 7 = -23 - 7'), Step(explanation='Simplify both sides of the equation. On the left, 7 and -7 cancel each other out, leaving 8x, and on the right, you perform the subtraction -23 - 7.', output='8x = -30'), Step(explanation='Now, solve for x by dividing both sides by 8 to isolate x.', output='x = -30 / 8'), Step(explanation='Simplify the right side by reducing the fraction -30/8 to its simplest form. Both the numerator and the denominator can be divided by their greatest common divisor, which is 2.', output='x = -15/4'), Step(explanation='Express the fraction as a decimal. Divide -15 by 4 to get the decimal equivalent.', output='x = -3.75')], final_answer='x = -3.75')

## Structured Output with Function Calling

Useful when you are connecting the model to tools, functions, data, etc. in your system.

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_delivery_date",
            "description": "Get the delivery date for a customer's order. Call this whenever you need to know the delivery date, for example when a customer asks 'Where is my package'",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The customer's order ID.",
                    },
                },
                "required": ["order_id"],
                "additionalProperties": False,
            },
        },
        "strict": True, # enables structured outputs
    }
]

messages = []
messages.append({"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."})
messages.append({"role": "user", "content": "Hi, can you tell me the delivery date for my order?"})

response = client.chat.completions.create(
    model='gpt-4o-2024-08-06',
    messages=messages,
    tools=tools,
)

In [ ]:
response

ChatCompletion(id='chatcmpl-9tJnKCqilscXhdn5omdhiEsKAUIEr', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Could you please provide me with your order ID so I can check the delivery date for you?', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1722971158, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_2a322c9ffc', usage=CompletionUsage(completion_tokens=20, prompt_tokens=107, total_tokens=127))

## Structured Output with `response_format`

Useful when the model needs to respond to the user in a specified structured way.

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
        {"role": "user", "content": "how can I solve 8x + 7 = -23"}
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "math_response",
            "schema": {
                "type": "object",
                "properties": {
                    "steps": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "prtropeies": {
                                "explanation": {"type": "string"},
                                "output": {"type": "string"}
                            },
                            "required": ["explanation", "output"],
                            "additionalProperties": False
                        }
                    },
                    "final_answer": {"type": "string"}
                },
                "required": ["steps", "final_answer"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
)

print(response.choices[0].message.content)

{"steps":[{"explanation":"The equation we start with is 8x + 7 = -23. To solve for x, we need to isolate x on one side of the equation.","output":"Start with 8x + 7 = -23."},{"explanation":"First, subtract 7 from both sides of the equation to begin isolating the term with x.","output":"8x + 7 - 7 = -23 - 7"},{"explanation":"After performing the subtraction, we simplify both sides to get rid of the 7 on the left. This gives us 8x on the left and -30 on the right.","output":"8x = -30"},{"explanation":"Now we need to solve for x by dividing both sides of the equation by 8.","output":"8x / 8 = -30 / 8"},{"explanation":"Simplify the right side to give us the value of x. -30 divided by 8 is -3.75.","output":"x = -3.75"}],"final_answer":"x = -3.75"}
